# Scrapping training data from Spotify API
## Spotify API login

In [ ]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

client_id = "078c39e69bf64995bb1bd3d6df37d557"
client_secret="855d6393e9e941f2aa359d8d58a9b1ff"
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

## Scrapping

In [ ]:
import os
import requests
from tqdm import tqdm

genres = spotify.recommendation_genre_seeds()["genres"]

tracks_previews = {}
offsets = [i for i in range(0, 1050, 50)]

# number of results is limited to 50 so we loop queries
for i, offset in enumerate(offsets):
    print(f"run {i+1}/{len(offsets)}")
    for genre in genres:
        print("processing", genre)
        list_by_genre= []
        tracks = spotify.search(q=genre, limit=50, offset=offset, type="track")
        for track in tracks["tracks"]["items"]:
            preview = track["preview_url"]
            if preview:       
                list_by_genre.append(track["preview_url"])
        tracks_previews[genre] = list_by_genre
        genre_dir = os.path.join("genres", genre)
        if not os.path.exists(genre_dir): os.makedirs(genre_dir)
        for url in tqdm(tracks_previews[genre]):
            i = len(os.listdir(genre_dir))
            r = requests.get(url)
            with open(os.path.join(genre_dir, str(i+1)) + ".mp3", "wb") as f:
                f.write(r.content)